In [5]:

!pip3 install -q -U langchain-openai langchain langgraph rich python-dotenv tavily-python  langchain_community langchain-tavily



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
import os 
from dotenv import load_dotenv
from typing import TypedDict,List,Annotated,Optional

#Langchain Components
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage,ToolMessage,HumanMessage,SystemMessage
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel,Field
from langchain_tavily import TavilySearch
#Langgraph cmponents
from langgraph.graph import StateGraph,END
from langgraph.graph.message import AnyMessage,add_messages
from langgraph.prebuilt import ToolNode,tools_condition

#For pretty printing
from rich.console import Console
from rich.markdown import Markdown
console = Console()



Phase 1 : A monolithic 'generalist Agent'

In [7]:
console=Console()

class AgentState(TypedDict):
    messages:Annotated[List[AnyMessage],add_messages]


search_tool=TavilySearch(max_results=2,name="web_search")
llm=ChatOpenAI(model='gpt-4o-mini',temperature=0)
llm_with_tools=llm.bind_tools([search_tool])

def monolithic_agent_node(state:AgentState):
    console.print("---Monolithic Agent: Thinking--")
    response=llm_with_tools.invoke(state['messages'])
    return {'messages':[response]}

tool_node=ToolNode([search_tool])

#Build the reAct graph for monolithic agent
mono_graph_builder=StateGraph(AgentState)
mono_graph_builder.add_node("agent", monolithic_agent_node)
mono_graph_builder.add_node("tools", tool_node)
mono_graph_builder.set_entry_point("agent")

def tools_condition_with_end(state):
    result = tools_condition(state)
    if isinstance(result, str):
        # Older versions return just "tools" or "agent"
        return {result: "tools", "__default__": END}
    elif isinstance(result, dict):
        # Newer versions return a mapping
        result["__default__"] = END
        return result
    else:
        raise TypeError(f"Unexpected type from tools_condition: {type(result)}")

mono_graph_builder.add_conditional_edges("agent", tools_condition_with_end)
mono_graph_builder.add_edge("tools", "agent")



monolithic_agent_app = mono_graph_builder.compile()

print("Monolithic 'generalist' agent compiled successfully.")

Monolithic 'generalist' agent compiled successfully.


In [8]:
company = "NVIDIA (NVDA)"
monolithic_query = f"Create a brief but comprehensive market analysis report for {company}. The report should include three sections: 1. A summary of recent news and market sentiment. 2. A basic technical analysis of the stock's price trend. 3. A look at the company's recent financial performance."

console.print(f"[bold yellow]Testing MONOLITHIC agent on a multi-faceted task:[/bold yellow]\n'{monolithic_query}'\n")

final_mono_output = monolithic_agent_app.invoke({
    "messages": [
        SystemMessage(content="You are a single, expert financial analyst. You must create a comprehensive report covering all aspects of the user's request."),
        HumanMessage(content=monolithic_query)
    ]
})

console.print("\n--- [bold red]Final Report from Monolithic Agent[/bold red] ---")
console.print(Markdown(final_mono_output['messages'][-1].content))

Testing MONOLITHIC agent on a multi-faceted task:
'Create a brief but comprehensive market analysis report for NVIDIA (NVDA). The report should include three 
sections: 1. A summary of recent news and market sentiment. 2. A basic technical analysis of the stock's price 
trend. 3. A look at the company's recent financial performance.'

---Monolithic Agent: Thinking--

Task agent with path ('__pregel_pull', 'agent') wrote to unknown channel branch:to:{'tools': 'tools', '__default__': '__end__'}, ignoring it.


--- Final Report from Monolithic Agent ---

# A Technical Analyst

In [15]:
class MultiAgentState(TypedDict):
    user_request:str
    news_report:Optional[str]
    technical_report:Optional[str]
    financial_report:Optional[str]
    final_report:Optional[str]

def create_specialist_node(persona:str, output_key:str):
    """Factory function to createa a specialist agent node"""
    system_prompt=persona+ "\n\n You have access to a web search tool. Your output must be a consise report section, formatted in markdown, focusing only on your area of expertise."

    prompt_template=ChatPromptTemplate.from_messages([("system",system_prompt),("human","{user_request}")])

    agent=prompt_template | llm_with_tools

    def specialist_node(state: MultiAgentState):
        console.print(f"---- Calling {output_key.replace('_report','').upper()} ANalyst--")
        result=agent.invoke({"user_request":state["user_request"]})
        content=result.content if result.content else f"No direct content, tool calls: {result.tool_calls}"
        return {output_key:content}
    
    return specialist_node
    
#create the specialist nodes
news_analyst_node=create_specialist_node(" You are an expert News Analyst. Your specialty is scouring the web for the latest news, articles, and social media sentiment about a company.",
    "news_report")

technical_analyst_node = create_specialist_node(
    "You are an expert Technical Analyst. You specialize in analyzing stock price charts, trends, and technical indicators.",
    "technical_report"
)
financial_analyst_node = create_specialist_node(
    "You are an expert Financial Analyst. You specialize in interpreting financial statements and performance metrics.",
    "financial_report"
)

def report_writer_node(state: MultiAgentState):
    """The manager agent that synthesizes the specialist reports"""
    console.print("CALLing report writer")
    prompt=f"""You are an expert financial editor. Your task is to combine the following specialist reports into a single, professional, and cohesive market analysis report. Add a brief introductory and concluding paragraph.
    
    News & Sentiment Report:
    {state['news_report']}

    Technical Analysis Report:
    {state['technical_report']}

    Financial Performance Report:
    {state['financial_report']}
    """

    final_report=llm.invoke(prompt).content
    return {'final_report': final_report}

print("specialist agent nodes and Reported writer node defined")


 



specialist agent nodes and Reported writer node defined


In [16]:
multi_agent_graph_builder=StateGraph(MultiAgentState)

multi_agent_graph_builder.add_node('news_analyst',news_analyst_node)
multi_agent_graph_builder.add_node('technical_analyst',technical_analyst_node)
multi_agent_graph_builder.add_node('financial_analyst',financial_analyst_node)
multi_agent_graph_builder.add_node('report_writer',report_writer_node)

multi_agent_graph_builder.set_entry_point("news_analyst")
multi_agent_graph_builder.add_edge('news_analyst','technical_analyst')
multi_agent_graph_builder.add_edge('technical_analyst','financial_analyst')
multi_agent_graph_builder.add_edge('financial_analyst','report_writer')
multi_agent_graph_builder.add_edge('report_writer',END)

multi_agent_app=multi_agent_graph_builder.compile()
print('Multi Agent graph built succesfully')




Multi Agent graph built succesfully


In [18]:
company='5c Network'
multi_agent_query=f"Create a brief but comprehensive market analysis report for {company}."
initial_multi_agent_input={'user_request':multi_agent_query}
console.print(f"[bold green] tetsing Multi Gnet team on the same atsk:[/bold green]\n '{multi_agent_graph_builder}'---")

final_multi_agent_output=multi_agent_app.invoke(initial_multi_agent_input)
console.print("\n--- [bold green]Final Report from Multi-Agent Team[/bold green] ---")
console.print(Markdown (final_multi_agent_output['final_report']))


 tetsing Multi Gnet team on the same atsk:
 '<langgraph.graph.state.StateGraph object at 0x1127b9390>'---

---- Calling NEWS ANalyst--

---- Calling TECHNICAL ANalyst--

---- Calling FINANCIAL ANalyst--

CALLing report writer

--- Final Report from Multi-Agent Team ---

Market Analysis Report: 5c Network                                                                                 

Introduction                                                                                                       

The 5c Network has garnered significant attention in the financial markets, reflecting a blend of investor         
sentiment, technical indicators, and financial performance metrics. This report synthesizes insights from various  
analyses to provide a comprehensive overview of the current market landscape for the 5c Network, highlighting key  
trends and potential future developments.                                                                          

News & Sentiment Analysis                                                                                          

While specific news content was not available for direct analysis, the overall sentiment surrounding the 5c Network
appears to be cautiously optimistic. Market participants are closely monitoring developments that could impact the 
network's growth and adoption. Investor sentiment is often influenced by broader market trends, regulatory updates,
and technological advancements within the blockchain and networking sectors.                                       

Technical Analysis                                                                                                 

The technical indicators for the 5c Network suggest a period of consolidation, with key support and resistance     
levels being tested. Traders are advised to watch for breakout patterns that could signal a shift in momentum. The 
analysis indicates that if the price can maintain above critical support levels, there may be potential for upward 
movement. Conversely, a failure to hold these levels could lead to increased volatility and downward pressure.     

Financial Performance Overview                                                                                     

The financial performance of the 5c Network remains a focal point for investors. While specific financial metrics  
were not detailed, the overall market performance suggests a stable growth trajectory. Investors are encouraged to 
consider the network's revenue streams, user adoption rates, and operational efficiencies as indicators of its     
long-term viability. Continued monitoring of these factors will be essential for assessing the network's financial 
health and potential for future growth.                                                                            

Conclusion                                                                                                         

In summary, the 5c Network is positioned within a dynamic market environment characterized by cautious optimism and
technical consolidation. As investors navigate this landscape, it is crucial to remain informed about market       
sentiment, technical indicators, and financial performance metrics. Ongoing analysis will be vital in identifying  
opportunities and risks associated with the 5c Network, ensuring that stakeholders can make informed decisions in  
this evolving market.

In [19]:
class ReportEvaluation(BaseModel):
    """Schema for Evaluating financial report"""
    clarity_and_structure_score:int =Field(description="Score 1-10 on the report's organization, structure, and clarity.")
    analytical_depth_core:int=Field(description='Score 1-10 on the depth and quality of the analysis in each section.')
    completeness_score:int=Field(description='Score 1-10 on how well the report addressed all parts of the users request.')
    justification :str=Field(description="A brief justification for the scores.")

judge_llm=llm.with_structured_output(ReportEvaluation)
                                 
def evaluate_report(query:str,report:str):
    prompt=f"""You are an expert judge of financial analysis reports. Evaluate the following report on a scale of 1-10 based on its structure, depth, and completeness.
    
    **Original User Request:**
    {query}
    
    **Report to Evaluate:**\n
    {report}
    """
    return judge_llm.invoke(prompt)

console.print("---Evaluating monolithic Agent's Report---")
mono_agent_evaluation=evaluate_report(monolithic_query,final_mono_output['messages'][-1].content)
console.print(mono_agent_evaluation.model_dump())

console.print("--EValuating multi agent's rpeort-- ")
multi_agent_evaluation=evaluate_report(multi_agent_query,final_multi_agent_output['final_report'])
console.print(multi_agent_evaluation.model_dump())



---Evaluating monolithic Agent's Report---

{
    'clarity_and_structure_score': 8,
    'analytical_depth_core': 7,
    'completeness_score': 9,
    'justification': "The report is well-structured with clear headings for each section, making it easy to follow.
The summary of recent news and market sentiment is concise and relevant, providing a good overview. The technical 
analysis includes key price trends and indicators, though it could benefit from more detailed explanations. The 
financial performance section covers essential metrics but lacks deeper insights into the implications of the 
numbers. Overall, it addresses the user's request comprehensively, but there is room for improvement in analytical 
depth."
}

--EValuating multi agent's rpeort--

{
    'clarity_and_structure_score': 7,
    'analytical_depth_core': 6,
    'completeness_score': 5,
    'justification': "The report is well-structured with clear sections, making it easy to follow. However, the 
depth of analysis in each section is somewhat lacking, particularly in the Financial Performance Overview where 
specific metrics are missing. The report addresses the user's request but does not provide comprehensive data or 
insights, which affects its completeness."
}